# Train a Chatbot Model
1. By the end of this notebook I should be able to:  
    a. Understand the process required to train a chatbot Model  
    b. Understand the required inputs to a chatbot Model  
    c. Understand the outputs of a chatbot Model  
    d. Know how to run a chatbot (from the command line)  
    e. Link a chatbot to a database (or at least a function that I can pass replies to)  
2. Stretch Goals  
    a. Build a small UI to run the chatbot in  

## Import Data
For this chatbot, I will use the taskmaster-2 dataset. It differs from taskmaster-1 in that it contains search and recommendation oriented dialogs, as well as task-based oriented dialogs, whereas taskmaster-1 was purely task-based. i.e. taskmaster-2 is more suitable for our purpose, since we are trying to build a restaurant recommender, as opposed to a restaurant reservation service.  

Starting with the user training data that was previously extracted

In [1]:
import json

# data_str = json.load(open('C:\\Users\\Ben\\Documents\\ebacs\\practical_language_processing\\ebacs_plp_project\\data\\user_training_data.json'))
data = json.load(open('C:\\Users\\Ben\\Documents\\ebacs\\practical_language_processing\\ebacs_plp_project\\data\\user_training_data.json'))

# data = json.loads(data_str)


In [2]:
data[0:3]

[['Hi, I need help finding a sandwich shop to go eat at.',
  {'entities': [[26, 34, 'restaurant.type.food']]}],
 ["Zoccoli's Delicatessen at 1534 Pacific Avenue.",
  {'entities': [[0, 22, 'restaurant.name.restaurant']]}],
 ['Next, The Sandwich Spot at 1010 Pacific Avenue.',
  {'entities': [[6, 23, 'restaurant.name.restaurant']]}]]

In [3]:
import spacy
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.blank('en')

if 'ner' not in nlp.pipe_names :
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else :
    ner = nlp.get_pipe("ner")

In [4]:
# Adding labels to the `ner`

for _, annotations in data:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [5]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [7]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):
  optimizer = nlp.begin_training()
  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(data)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(data, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

5.417536883779}
Losses {'ner': 13528.858296959825}
Losses {'ner': 13555.258848636575}
Losses {'ner': 13571.182482688851}
Losses {'ner': 13582.792217939324}
Losses {'ner': 13596.165571688838}
Losses {'ner': 13605.873706101604}
Losses {'ner': 13616.32800138665}
Losses {'ner': 13624.084302610881}
Losses {'ner': 13628.488700806327}
Losses {'ner': 13649.958231031127}
Losses {'ner': 13663.120840489097}
Losses {'ner': 13686.155863463111}
Losses {'ner': 13704.65247243566}
Losses {'ner': 13720.504880770273}
Losses {'ner': 13732.744878753252}
Losses {'ner': 13743.091487541027}
Losses {'ner': 13747.903209461994}
Losses {'ner': 13766.487396731205}
Losses {'ner': 13777.847793295212}
Losses {'ner': 13804.063526346512}
Losses {'ner': 13823.313977196045}
Losses {'ner': 13837.071679189034}
Losses {'ner': 13856.83599765522}
Losses {'ner': 13873.014409735031}
Losses {'ner': 13882.853473155923}
Losses {'ner': 13906.435965030618}
Losses {'ner': 13941.635625331826}
Losses {'ner': 13955.385388522573}
Losses 

In [11]:
# Testing the model
doc = nlp("I am looking for a pizza spot for dinner")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('pizza', 'restaurant.type.food')]


In [15]:
# Output directory
from pathlib import Path
output_dir=Path('C:\\Users\\Ben\\Documents\\ebacs\\practical_language_processing\\ebacs_plp_project\\model\\')

# Saving the model to the output directory
if not output_dir.exists():
  output_dir.mkdir()
nlp.meta['name'] = 'chatbot_ner'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to C:\Users\Ben\Documents\ebacs\practical_language_processing\ebacs_plp_project\model


In [31]:
for ent in doc.ents:
    print(ent.label_)

restaurant.type.food


In [38]:
from spacy import displacy

displacy.render(doc, style = "ent")